The idea behind this notebook is to make an analysis on the CoronaVirus dataset using the package Prophet

I am fairly new time-series analysis and I wanted to test how the algorithms provided by the package work.

In [1]:
import os

os.chdir(r'C:\Users\p.tatti.AVANADE-CORP\Desktop\Kaggle\CoronaVirus')
os.listdir()

['.ipynb_checkpoints',
 '2019-coronavirus-dataset-01212020-01262020.zip',
 '2019_nC0v_20200121_20200126 - SUMMARY.csv',
 '2019_nC0v_20200121_20200126_cleaned.csv',
 '2019_nCoV_20200121_20200127.csv',
 '2019_nCoV_20200121_20200128.csv',
 '2019_nCoV_20200121_20200129.csv',
 '2019_nCoV_data.csv',
 'ciao.csv',
 'CoronaVirus.ipynb',
 'COVID-19-master',
 'COVID-19-master.zip',
 'indirizzi.xlsx',
 'nCov_cumulative.csv',
 'time_series_2019-ncov.xlsx',
 'time_series_2019-ncov_6_2_20.xlsx',
 'Untitled.ipynb']

Load libraries and some utility functions

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
plt.style.use('ggplot')


import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.figure_factory as ff
from plotly import subplots
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)


from datetime import datetime, date, timedelta

from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')



pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
def get_data(path):
    tab_name = ['Deaths','Recovered']
    jc = ['Province/State','Country/Region']
    path_c = path+'Confirmed.csv'
    df = pd.read_csv(path_c)
    df = pd.melt(df, id_vars=['Province/State','Country/Region','Lat','Long'],
            var_name='Date', value_name= 'Confirmed')
    df['Date'] = pd.to_datetime(df['Date'])
    for name in tab_name:
        path_ = path+name+'.csv'
        data = pd.read_csv(path_)
        data = pd.melt(data, id_vars=['Province/State','Country/Region','Lat','Long'], 
                    var_name='Date', value_name= name)
        data['Date'] = pd.to_datetime(data['Date'])
        df[name] = data[name].values
        
        
    return(df)

def prepare_data(df):
    num_col = ['Confirmed','Deaths','Recovered'] 
    new_col = ['PS','Country','Lat','Long','Date','Confirmed','Deaths','Recovered']
    df.columns = new_col
    df[num_col] = df[num_col].apply(lambda x: x.fillna(value = 0))
    df[num_col] = df[num_col].astype(np.int32)
    df['Country'] = np.where(df['Country'] == 'Mainland China','China',df['Country'])
    df['PS'] = np.where(df['PS'].isnull(), df['Country'],df['PS'])
    
    return(df)
 
def check_anomalies(df):
    count_c = df.loc[(df['Confirmed_'] <0)].shape[0]
    count_d = df.loc[(df['Deaths_'] <0)].shape[0]
    count_r = df.loc[(df['Recovered_'] <0)].shape[0]
    
    print("Number of negative Confirmed_: {}\n".format(count_c))
    print("Number of negative Deaths_: {}\n".format(count_d))
    print("Number of negative Recovered_: {}\n".format(count_r))
    

I am using the data extracted from the Time series google sheet, now on github https://github.com/CSSEGISandData/COVID-19/tree/master/time_series

In [4]:
path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-'
df = get_data(path)

The **get_data** function read the data from the three different csv files, saves the first columns of the table and unpivots the last column in order to retrieve the cumulate distribution of **Confirmed**, **Deaths** and **Recovered** found on different files.

In [5]:
df.head(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.82571,117.2264,2020-01-21 22:00:00,NaN,NaN,NaN
1,Beijing,Mainland China,40.18238,116.4142,2020-01-21 22:00:00,10.0,NaN,NaN
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-21 22:00:00,5.0,NaN,NaN
3,Fujian,Mainland China,26.07783,117.9895,2020-01-21 22:00:00,NaN,NaN,NaN
4,Gansu,Mainland China,36.06110,103.8343,2020-01-21 22:00:00,NaN,NaN,NaN
5,Guangdong,Mainland China,23.33841,113.4220,2020-01-21 22:00:00,17.0,NaN,NaN
6,Guangxi,Mainland China,23.82908,108.7881,2020-01-21 22:00:00,NaN,NaN,NaN
7,Guizhou,Mainland China,26.81536,106.8748,2020-01-21 22:00:00,NaN,NaN,NaN
8,Hainan,Mainland China,19.19673,109.7455,2020-01-21 22:00:00,NaN,NaN,NaN
9,Hebei,Mainland China,38.04280,114.5149,2020-01-21 22:00:00,NaN,NaN,NaN


Since there were several NaN values in the last three columns, I decided to fill them with $0$, because they were the starting point of the cumulates. Moreover, I filled the **NaN** values in the **Province/State** column, entering the Country if the value was missing and changed the value Mainland China to China for semplicity.
Furthermore I renamed the columns and cast some columns to the correct type.

In [6]:
df = prepare_data(df)

In [7]:
df.head(10)

,PS,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,China,31.82571,117.2264,2020-01-21 22:00:00,0,0,0
1,Beijing,China,40.18238,116.4142,2020-01-21 22:00:00,10,0,0
2,Chongqing,China,30.05718,107.8740,2020-01-21 22:00:00,5,0,0
3,Fujian,China,26.07783,117.9895,2020-01-21 22:00:00,0,0,0
4,Gansu,China,36.06110,103.8343,2020-01-21 22:00:00,0,0,0
5,Guangdong,China,23.33841,113.4220,2020-01-21 22:00:00,17,0,0
6,Guangxi,China,23.82908,108.7881,2020-01-21 22:00:00,0,0,0
7,Guizhou,China,26.81536,106.8748,2020-01-21 22:00:00,0,0,0
8,Hainan,China,19.19673,109.7455,2020-01-21 22:00:00,0,0,0
9,Hebei,China,38.04280,114.5149,2020-01-21 22:00:00,0,0,0


In [8]:
df.isnull().sum()

PS           0
Country      0
Lat          0
Long         0
Date         0
Confirmed    0
Deaths       0
Recovered    0
dtype: int64

In [9]:
print("Number of rows in the dataset: {}".format(df.shape[0]))
print("Number of Columns in the dataset: {}".format(df.shape[1]))

Number of rows in the dataset: 3256
Number of Columns in the dataset: 8


## Exploratory Data Analysis

I want to show how each day gone by the number of **Confirmed**,**Deaths** and **Recovered** changed, so I created a new Dataframe **sorted_df** in which I had $3$ columns, containing the number of cases, for each category, happened during that day.

In [10]:
sorted_df = df.sort_values(['Country','PS', 'Date'])
cols = ['Country', 'PS']
sorted_df['Confirmed_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1)
                                   , sorted_df['Confirmed'].diff(), sorted_df['Confirmed'])
sorted_df['Deaths_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1),
                                sorted_df['Deaths'].diff(), sorted_df['Deaths'])
sorted_df['Recovered_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1),
                                   sorted_df['Recovered'].diff(), sorted_df['Recovered'])

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Now I want to check if the computation of the daily **Confirmed**, **Deaths** and **Recovered** is correct, i.e if I only get positive values.

In [11]:
check_anomalies(sorted_df)

Number of negative Confirmed_: 3

Number of negative Deaths_: 0

Number of negative Recovered_: 12



In [12]:
sorted_df.loc[sorted_df['Confirmed_']<0]

,PS,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Confirmed_,Deaths_,Recovered_
1316,India,India,20.5937,78.9629,2020-01-29 14:30:00,0,0,0,-1.0,0.0,0.0
2252,Japan,Japan,35.6762,139.6503,2020-02-07 20:13:00,25,0,1,-20.0,0.0,0.0
2401,South Korea,South Korea,37.5665,126.9780,2020-02-08 10:24:00,24,0,1,-12.0,0.0,0.0


In [13]:
# japan
df.loc[[2030,2104,2178],'Confirmed'] = 22
# south Korea
df.loc[[2327],'Confirmed'] = 24
# india
df.loc[[1316,1390],'Confirmed'] = 1

In [14]:
sorted_df.loc[sorted_df['Recovered_']<0]

,PS,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Confirmed_,Deaths_,Recovered_
1186,Chongqing,China,30.05718,107.8740,2020-01-29 13:30:00,147,0,0,0.0,0.0,-1.0
2892,Guangxi,China,23.82908,108.7881,2020-02-11 20:44:00,222,1,31,7.0,0.0,-2.0
2005,Guizhou,China,26.81536,106.8748,2020-02-05 23:00:00,69,1,6,5.0,0.0,-3.0
1266,Hainan,China,19.19673,109.7455,2020-01-29 14:30:00,43,1,0,0.0,0.0,-1.0
2822,Heilongjiang,China,47.86200,127.7622,2020-02-11 10:50:00,360,8,28,29.0,1.0,-2.0
3197,Jiangsu,China,32.97027,119.4640,2020-02-13 21:15:00,593,0,137,23.0,0.0,-2.0
2461,Ningxia,China,37.26923,106.1655,2020-02-08 23:04:00,45,0,13,0.0,0.0,-2.0
3055,Shaanxi,China,35.19165,108.8701,2020-02-12 22:00:00,229,0,42,4.0,0.0,-1.0
2317,Shanghai,China,31.20327,121.4554,2020-02-07 22:50:00,281,1,3,4.0,0.0,-27.0
1726,Shanxi,China,37.57769,112.2922,2020-02-03 21:00:00,74,0,2,8.0,0.0,-1.0


In [15]:
# Chongqing
df.loc[1186,'Recovered'] = 1
# Guangxi
df.loc[2818,'Recovered'] = 31
# Guizhou
df.loc[1931,'Recovered'] = 5
#df.loc[2926,'Recovered'] = 33
# Hainan
df.loc[[1266,1340],'Recovered'] = 1
# Heilongjiang
df.loc[[2674,2748],'Recovered'] = 22
#Jiangsu
df.loc[3197,'Recovered'] = 139
# Ningxia
df.loc[2387,'Recovered'] = 13
# Shanghai
df.loc[2317,'Recovered'] = 30
# Shanxi
df.loc[1652,'Recovered'] = 1
# Shaanxi
df.loc[2981,'Recovered'] = 42
# Sichuan
#df.loc[2288,'Recovered'] = 45
df.loc[2319,'Recovered'] = 50
# Yunnan
df.loc[2989,'Recovered'] = 23

As it's shown in previous cells, the values are incorrect. I changed manually, by leaving the value of the previous record. Then I created again the **sorted_df**

In [16]:
sorted_df = df.sort_values(['Country','PS', 'Date'])
cols = ['Country', 'PS']
sorted_df['Confirmed_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1)
                                   , sorted_df['Confirmed'].diff(), sorted_df['Confirmed'])
sorted_df['Deaths_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1),
                                sorted_df['Deaths'].diff(), sorted_df['Deaths'])
sorted_df['Recovered_'] = np.where(np.all(sorted_df.shift()[cols]==sorted_df[cols], axis=1),
                                   sorted_df['Recovered'].diff(), sorted_df['Recovered'])

In [17]:
check_anomalies(sorted_df)

Number of negative Confirmed_: 0

Number of negative Deaths_: 0

Number of negative Recovered_: 0



Now that data seems to be clean, I will start with some Exploratory Data Analysis.

## Exploratory Data Analysis

In [18]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=sorted_df.Country.loc[sorted_df['Country'] != 'China'],
        y=sorted_df.Confirmed_.loc[sorted_df['Country'] != 'China'],
        name='Rest of the world',
        marker_color='rgb(55, 83, 109)',
        text = sorted_df.Date.astype(str)
    )
)

fig.update_layout(
    title={'text': 'Confirmed case all over the world',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(tickangle=45),
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.1,
    bargroupgap=0.1
)
fig.show()

By moving on the plot, it is shown how many cases happened during a specific day. It's easy to notice the abundance of cases mark as *Others* that were only confirmed in $3$ days. As far as the other contries, they did not have the same behaviour.

In [19]:
fig = go.Figure()
fig.add_trace(go.Bar(x=sorted_df.PS.loc[sorted_df['Country'] == 'China'],
                y=sorted_df.Confirmed_.loc[sorted_df['Country'] == 'China'],
                name='China',
                marker_color='rgb(26, 118, 255)',
                text = sorted_df.loc[sorted_df['Country'] == 'China'].Date.astype(str)
                ))

fig.update_layout(
    title={'text': 'Confirmed case in China',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(tickangle=45),
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=sorted_df.PS.loc[(sorted_df['Country'] == 'China') & (sorted_df['PS'] != 'Hubei')],
                y=sorted_df.Confirmed_.loc[(sorted_df['Country'] == 'China') & (sorted_df['PS'] != 'Hubei')],
                name='China',
                marker_color='rgb(26, 118, 255)',
                text = df.Date.loc[(sorted_df['Country'] == 'China') & (sorted_df['PS'] != 'Hubei')].astype(str)
                ))

fig2.update_layout(
    title={'text': 'Confirmed case in China (not Hubei)',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(tickangle=45),
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig2.show()


Above you can see two barplots with only confirmed case in China. I decided to use 2 reprentation because the cases in the *Hubei* region are simply too much with respect to the others.

In [20]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=sorted_df.Country.loc[sorted_df['Country'] != 'China'],
        y=sorted_df.Deaths_.loc[sorted_df['Country'] != 'China'],
        name='Deaths',
        marker_color='rgb(55, 83, 109)',
        text = sorted_df.Date.astype(str)
    )
)
fig.add_trace(
    go.Bar(
        x=sorted_df.Country.loc[sorted_df['Country'] != 'China'],
        y=sorted_df.Recovered_.loc[sorted_df['Country'] != 'China'],
        name='Recovered',
        marker_color='rgb(26, 118, 255)',
        text = sorted_df.Date.astype(str)
    )
)
fig.update_layout(
    title={'text': 'Deaths & Recovered case all over the world',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(tickangle=45),
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1 
)
fig.show()

df_not_hubei = sorted_df.loc[sorted_df['PS'] != 'Hubei']


fig2 = go.Figure()
fig2.add_trace(
    go.Bar(
        x=df_not_hubei.PS.loc[df_not_hubei['Country'] == 'China'],
        y=df_not_hubei.Deaths_.loc[df_not_hubei['Country'] == 'China'],
        name='Deaths',
        marker_color='rgb(55, 83, 109)',
        text = sorted_df.Date.astype(str)
    )
)
fig2.add_trace(
    go.Bar(
        x=df_not_hubei.PS.loc[df_not_hubei['Country'] == 'China'],
        y=df_not_hubei.Recovered_.loc[df_not_hubei['Country'] == 'China'],
        name='Recovered',
        marker_color='rgb(26, 118, 255)',
        text = sorted_df.Date.astype(str)
    )
)
fig2.update_layout(
    title={'text': 'Deaths & Recovered case in China (not Hubei)',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(tickangle=45),
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14
        ,range = [0, df_not_hubei['Recovered'].max() + 10]
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1 
)
fig2.show()

Above you can see the same barplot, but this time showing the **Deaths** and **Recovered** cases.

In [21]:
df_hubei = sorted_df.loc[sorted_df['PS'] == 'Hubei']

fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(
        x=df_hubei.Date,
        y=df_hubei.Deaths,
        name='Deaths',
        mode='lines+markers',
        marker_color='rgb(55, 83, 109)'
    )
)

fig2.add_trace(
    go.Scatter(
        x=df_hubei.Date,
        y=df_hubei.Recovered,
        name='Recovered',
        marker_color='rgb(26, 118, 255)'
    )
)

fig2.update_traces(
    mode='lines+markers',
    marker_line_width=2,
    marker_size=5
)

fig2.update_layout(
    title={'text': 'Deaths and Recovered in Hubei (China)',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    yaxis_zeroline=False,
    xaxis_zeroline=False
)

fig2.show()

And now the plot of confirmed cases for the *Hubei* region. AS before that magnitude of the data is really different from the other, so i decided to slit and represent it only in one plot.

In [22]:
df_hubei['confirmed_case_world'] = df_not_hubei.groupby('Date').sum()['Confirmed'].values

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df_hubei.Date,
    y = df_hubei.Confirmed,
    name = 'Hubei',
    mode = 'lines+markers',
    marker_color = 'rgb(55,83,109)'))

fig.add_trace(
    go.Scatter(
        x=df_hubei.Date,
        y=df_hubei.confirmed_case_world,
        name='Other',
        marker_color='rgb(26, 118, 255)'
    )
)

fig.update_traces(mode='lines+markers',
                  marker_line_width=2,
                  marker_size=5)
fig.update_layout(
    title={'text': 'Confermed case in Hubei vs Rest of World',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    yaxis_zeroline=False,
    xaxis_zeroline=False
)

fig.show()

By looking at the plot, it is shown the difference between the confirmed cases of *Hubei* against the rest of the world.

Hence I decided to use only the data related to the *Hubei* region in my Predictive Analisys.

## Time Series Analysis - Prophet

In [23]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric, add_changepoints_to_plot, plot_plotly

In [24]:
df_prophet = df_hubei[['Date','Confirmed']]
df_prophet.columns = ['ds','y']

In order to find the best model, I decided to perform my test on some dates, leaving the last as test set.

### Basic model

Let's start by modeling a baseline model, including the daily trend. I do not think that this would be useful since the hour in the feature **Date** are not the real one in which the new confirmed case is registered.

In [25]:
m_d = Prophet(
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = 2,
    seasonality_mode = 'additive')
m_d.fit(df_prophet)
future_d = m_d.make_future_dataframe(periods=7)
fcst_daily = m_d.predict(future_d)

In [26]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": df_prophet.ds, 
  "y": df_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines", 
  "name": "prediction", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat
}
data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Confirmed - Time Series Forecast - Daily Trend", 
  "xaxis": {
    "title": "Monthly Dates", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Confirmed nCov - Hubei", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "paper_bgcolor": "rgb(243, 243, 243)"
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Let's try now by removing the *daily_seasonality*

In [27]:
m_nd = Prophet(
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = False,
    seasonality_mode = 'additive')
m_nd.fit(df_prophet)
future_nd = m_nd.make_future_dataframe(periods=7)
fcst_no_daily = m_nd.predict(future_nd)

In [28]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": df_prophet.ds, 
  "y": df_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines", 
  "name": "prediction", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat
}
data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Confirmed - Time Series Forecast", 
  "xaxis": {
    "title": "Monthly Dates", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Confirmed nCov - Hubei", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "paper_bgcolor": "rgb(243, 243, 243)"
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Let's see how the two models perform in terms of Mean absolute percentage Error

In [29]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [30]:
max_date = df_prophet.ds.max()

In [31]:
y_true = df_prophet.y.values
y_pred_daily = fcst_daily.loc[fcst_daily['ds'] <= max_date].yhat.values
y_pred_no_daily = fcst_no_daily.loc[fcst_daily['ds'] <= max_date].yhat.values

In [32]:
print('MAPE with daily seasonality: {}'.format(mean_absolute_percentage_error(y_true,y_pred_daily)))
print('MAPE without daily seasonality: {}'.format(mean_absolute_percentage_error(y_true,y_pred_no_daily)))

MAPE with daily seasonality: 78.486108865299
MAPE without daily seasonality: 74.39090926720245


It is pretty clear that the models perform very bad

Let's try to add some parameters into the both model and see if something changes, hoping for an improvement.

In [33]:
m_d = Prophet(
    changepoint_prior_scale=20,
    seasonality_prior_scale=20,
    n_changepoints=10,
    changepoint_range=0.9,
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = True,
    seasonality_mode = 'additive')
m_d.fit(df_prophet)
future_d = m_d.make_future_dataframe(periods=7)
fcst_daily = m_d.predict(future_d)

In [34]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": df_prophet.ds, 
  "y": df_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines", 
  "name": "prediction", 
  "type": "scatter", 
  "x": fcst_daily.ds, 
  "y": fcst_daily.yhat
}
data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Confirmed - Time Series Forecast - Daily Trend", 
  "xaxis": {
    "title": "Monthly Dates", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Confirmed nCov - Hubei", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "paper_bgcolor": "rgb(243, 243, 243)"
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [35]:
m_nd = Prophet(
    changepoint_range=0.90,
    changepoint_prior_scale=20,
    n_changepoints=20,
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = False,
    seasonality_mode = 'additive')
m_nd.fit(df_prophet)
future_nd = m_nd.make_future_dataframe(periods=7)
fcst_no_daily = m_nd.predict(future_nd)

In [36]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": df_prophet.ds, 
  "y": df_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines", 
  "name": "prediction", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat
}
data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Confirmed - Time Series Forecast", 
  "xaxis": {
    "title": "Monthly Dates", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Confirmed nCov - Hubei", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "paper_bgcolor": "rgb(243, 243, 243)"
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [37]:
y_true = df_prophet.y.values
y_pred_daily = fcst_daily.loc[fcst_daily['ds'] <= max_date].yhat.values
y_pred_no_daily = fcst_no_daily.loc[fcst_daily['ds'] <= max_date].yhat.values

In [38]:
print('MAPE with daily seasonality: {}'.format(mean_absolute_percentage_error(y_true,y_pred_daily)))
print('MAPE without daily seasonality: {}'.format(mean_absolute_percentage_error(y_true,y_pred_no_daily)))

MAPE with daily seasonality: 12.421153892799177
MAPE without daily seasonality: 6.570174001648763


The changes made seem to have brought a significant improvement on both models. I try randomly to change the paramenters related to **changepoints** on both models. Obviously by incrementing the **prior_scale** we can get a more flexible model, which brought the major improvements.

About the number of **changepoint** to pass to the model, you can look at the plot below.

In [39]:
df_ch = pd.DataFrame()

df_ch['deltas'] = m_nd.params['delta'].mean(0)
df_ch['x'] = [x for x in range(20)]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=df_ch.x,
        y=df_ch.deltas,
        name='# of changepoints',
        marker_color='rgb(55, 83, 109)',
        text = df_ch.deltas
    )
)

fig.update_layout(
    title={'text': 'Barplot of ChangePoints',
           'y':0.95,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_tickfont_size=14,
    xaxis=dict(
        title = 'Potential ChangePoint'),
    yaxis=dict(
        title='Rate Change',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.1,
    bargroupgap=0.1
)
fig.show()

## Deaths predicitons

In [40]:
df_prophet = df_hubei[['Date','Deaths']]
df_prophet.columns = ['ds','y']

In [66]:
m_nd = Prophet(
    changepoint_range=0.90,
    changepoint_prior_scale=20,
    n_changepoints=20,
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = False,
    seasonality_mode = 'additive')
m_nd.fit(df_prophet.loc[df_prophet['ds']< pd.to_datetime('2020-02-10')])
future_nd = m_nd.make_future_dataframe(periods=3,freq = '12H')
fcst_no_daily = m_nd.predict(future_nd)

In [67]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": df_prophet.ds, 
  "y": df_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines", 
  "name": "prediction", 
  "type": "scatter", 
  "x": fcst_no_daily.ds, 
  "y": fcst_no_daily.yhat
}
data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Deaths - Time Series Forecast", 
  "xaxis": {
    "title": "Monthly Dates", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Deaths nCov - Hubei", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "paper_bgcolor": "rgb(243, 243, 243)"
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Since there are some $0$ value in the **Deaths** feature, I couldn't use the **MAPE** as before, I decided to use the **Mean Absolute Scaled Error**.

In [68]:
def MASE(training_series, testing_series, prediction_series):
    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d

In [72]:
max_date = pd.to_datetime('2020-02-10')
y_train = df_prophet.y.loc[df_prophet['ds'] < max_date].values
y_test =  df_prophet.y.loc[df_prophet['ds'] >= max_date].values
y_pred_deaths = fcst_no_daily.loc[fcst_no_daily['ds'] <= max_date].yhat.values

print('MASE without daily seasonality: {}'.format(MASE(y_train,y_test,y_pred_deaths)))

ValueError: operands could not be broadcast together with shapes (8,) (36,) 

In [71]:
pd.to_datetime('2020-02-10')

Timestamp('2020-02-10 00:00:00')

In [70]:
fcst_no_daily

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-21 22:00:00,-0.215100,-17.306963,15.816687,-0.215100,-0.215100,0.0,0.0,0.0,0.0,0.0,0.0,-0.215100
1,2020-01-22 12:00:00,0.157806,-16.248850,16.626875,0.157806,0.157806,0.0,0.0,0.0,0.0,0.0,0.0,0.157806
2,2020-01-23 12:00:00,0.797073,-15.877093,17.554341,0.797073,0.797073,0.0,0.0,0.0,0.0,0.0,0.0,0.797073
3,2020-01-24 00:00:00,21.557050,3.916049,38.309821,21.557050,21.557050,0.0,0.0,0.0,0.0,0.0,0.0,21.557050
4,2020-01-24 12:00:00,26.666650,10.366590,43.123858,26.666650,26.666650,0.0,0.0,0.0,0.0,0.0,0.0,26.666650
5,2020-01-25 00:00:00,31.776249,13.938226,49.141813,31.776249,31.776249,0.0,0.0,0.0,0.0,0.0,0.0,31.776249
6,2020-01-25 12:00:00,41.470789,25.502618,58.667794,41.470789,41.470789,0.0,0.0,0.0,0.0,0.0,0.0,41.470789
7,2020-01-25 22:00:00,47.406458,31.663617,63.975859,47.406458,47.406458,0.0,0.0,0.0,0.0,0.0,0.0,47.406458
8,2020-01-26 11:00:00,55.122828,37.799817,71.459942,55.122828,55.122828,0.0,0.0,0.0,0.0,0.0,0.0,55.122828
9,2020-01-26 23:00:00,73.953323,57.403070,90.020943,73.953323,73.953323,0.0,0.0,0.0,0.0,0.0,0.0,73.953323


In [73]:
df_prophet

,ds,y
12,2020-01-21 22:00:00,0
86,2020-01-22 12:00:00,0
160,2020-01-23 12:00:00,0
234,2020-01-24 00:00:00,24
308,2020-01-24 12:00:00,24
382,2020-01-25 00:00:00,32
456,2020-01-25 12:00:00,40
530,2020-01-25 22:00:00,52
604,2020-01-26 11:00:00,52
678,2020-01-26 23:00:00,76
